In [ ]:
from __future__ import print_function, division
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import pathlib
import torch
import torchvision
import tqdm

import dsbtorch
import dsbfetch

plt.ion()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset_dir = pathlib.Path("/home/ubuntu/data/encoded_dataset/test")
dataset = dsbtorch.PreEmbeddedDataset(dataset_dir)

In [ ]:
def eval_model(model):
    model.eval()
    
    ious = []
    
    with torch.set_grad_enabled(False):
        for cnn_outputs, labels in tqdm.tqdm(dataset):
            probs = model(cnn_outputs.to(device))
            preds = torch.argmax(probs, dim=1)

            ious.append(dsbtorch.compute_IOU(preds.cpu(), labels))
                        
    return ious

In [ ]:
model = dsbtorch.ResCNNClassifier("/home/ubuntu/data/DeepSponsorBlock/results/rescnn.weights")

In [ ]:
# Since we're going to run this directly on the preprocessed files, we just need the FC layer.
classifier = model.fc
classifier = classifier.to(device)

In [ ]:
ious = eval_model(classifier)

In [ ]:
# Load labels from SponsorBlock segments file.
labeled_videos = dsbfetch.load_segments("../segments.csv")
labels = {x.video_id: x.segments for x in labeled_videos}

In [ ]:
# Compile the videos we evaluated
videos = [emb_file.stem.split('.')[0] for emb_file, _ in dataset.videos]
pairs = sorted(zip(videos, ious), key=lambda x: -x[1])[:50]

In [ ]:
# Get YouTube links
links = [("https://www.youtube.com/watch?v=%s" % (vid), iou, labels[vid]) for vid, iou in pairs]
print("\n".join(str(x) for x in links))

In [ ]:
print("Mean:", np.mean(ious))
print("Median:", np.median(ious))

In [ ]:
# Plot the histogram of IOUs
plt.figure(dpi=300)
plt.style.use('grayscale')

plt.hist(ious, bins=25)

title = "Number of Videos by IOU"
plt.title(title)

plt.show()